In [139]:
"""
 Peak alignment between mass spectrum in the whole dataset 
"""
import os
import pandas as pd
import pickle
import biotype.similarity as sim

In [140]:
DATA_DIR_PATH = "datasets/export"
META_FILE_PATH = "datasets/meta.csv"
PEAK_DIR_PATH = "extracted_peaks"
files =  os.listdir(DATA_DIR_PATH)
files = [fn  for fn in files if fn.endswith(".txt")]
files = sorted(files)
for file in files[:5] :
    print(file)

L100_0_G7_1.txt
L100_0_G8_1.txt
L101_0_A1_1.txt
L101_0_A2_1.txt
L102_0_A3_1.txt


In [141]:
n = len(files)
all_peaks = [] # list of tuples (m/z, intensity, dataset_id)
for i in range(n):
    df = None
    df = pd.read_table(f"{DATA_DIR_PATH}/{files[i]}",sep=" ", header=None,names=['m/z', 'intensity']) 
    x, y = df['m/z'].to_numpy(), df['intensity'].to_numpy()

    pickle_file = files[i][:-4] # trim '.txt'
    with open(f'{PEAK_DIR_PATH}/{pickle_file}_peaks.pkl', 'rb') as peak_file:
        peaks = pickle.load(peak_file)
        # peaks_extracted +=  [(x[peaks], y[peaks])] 
        for p in peaks:
            all_peaks += [(x[p],y[p], i)]

all_peaks = sorted(all_peaks, key=lambda x: x[0])
print(all_peaks[-20:])

[(20068.12, 169, 24), (20068.12, 328, 27), (20068.547, 180, 69), (20068.973, 469, 29), (20068.973, 184, 35), (20068.973, 523, 52), (20069.4, 284, 9), (20069.4, 463, 17), (20069.4, 282, 38), (20069.4, 216, 72), (20069.4, 129, 74), (20069.4, 402, 88), (20069.579, 72, 43), (20069.827, 150, 59), (20070.254, 261, 23), (20070.254, 284, 47), (20070.681, 481, 7), (20071.48, 1, 0), (20071.961, 264, 61), (20073.668, 246, 82)]


In [142]:
import itertools

def pairwise(iterable,):
    "s -> (s0, s1), (s1, s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a,b)

print (list(pairwise([1,2,3,5,6])))

[(1, 2), (2, 3), (3, 5), (5, 6)]


In [154]:

def align(peaks, delta=3):
    """partition peaks to sections whose m/z values approximately equal to each other 
    """
    div = [] 
    k = -1
    for a, b in pairwise(peaks): # partition peaks into sections when there is a gap bigger than delta
        k = k + 1
        if abs(a[0] - b[0]) > delta:
            div +=[k]
    
    new_div = [-1] + div[:]
    for a, b in pairwise(div):
        if b > a + 1 and abs(peaks[a+1][0] - peaks[b][0]) > delta:
            i = a + 1
            for j in range(i, b):
                if abs(peaks[i][0] - peaks[j+1][0]) > delta:
                    i = j+1
                    new_div += [j]

    return sorted(list(set(new_div)))
    

In [164]:
div = align(all_peaks, 5)
print(div[:10])
print(div[-10:])

[-1, 32, 106, 120, 164, 167, 219, 240, 321, 356]
[20820, 20822, 20825, 20834, 20856, 20866, 20867, 20869, 20871, 20872]


In [167]:
peak_id = 1
aligned_peaks = dict()
for a, b in pairwise(div):
    for mz, intensity, ms in all_peaks[a+1:b+1]:
        aligned_peaks[(mz, ms)] = peak_id
    if a+1 < b:
        peak_id = peak_id + 1

for a in aligned_peaks:
    print(a, ': ', aligned_peaks[a])

(1000.748, 42) :  1
(1000.748, 90) :  1
(1000.871, 93) :  1
(1000.871, 95) :  1
(1000.948, 100) :  1
(1001.081, 22) :  1
(1001.083, 0) :  1
(1001.273, 1) :  1
(1001.366, 44) :  1
(1001.366, 75) :  1
(1001.366, 83) :  1
(1001.461, 17) :  1
(1001.461, 46) :  1
(1001.461, 49) :  1
(1001.461, 68) :  1
(1001.556, 15) :  1
(1001.556, 26) :  1
(1001.556, 35) :  1
(1001.556, 45) :  1
(1001.556, 56) :  1
(1001.556, 72) :  1
(1001.556, 74) :  1
(1001.651, 5) :  1
(1001.651, 14) :  1
(1001.651, 54) :  1
(1001.746, 4) :  1
(1001.746, 7) :  1
(1001.746, 27) :  1
(1001.746, 29) :  1
(1001.936, 40) :  1
(1001.936, 50) :  1
(1001.936, 66) :  1
(1002.126, 73) :  1
(1007.214, 43) :  2
(1007.309, 20) :  2
(1007.405, 87) :  2
(1007.5, 21) :  2
(1007.5, 91) :  2
(1007.595, 99) :  2
(1007.716, 97) :  2
(1007.737, 24) :  2
(1007.832, 6) :  2
(1007.881, 42) :  2
(1007.881, 90) :  2
(1007.932, 0) :  2
(1008.023, 22) :  2
(1008.023, 57) :  2
(1008.023, 68) :  2
(1008.023, 75) :  2
(1008.023, 77) :  2
(1008.118,

(1166.412, 13) :  20
(1166.412, 35) :  20
(1166.412, 48) :  20
(1166.412, 54) :  20
(1166.412, 57) :  20
(1166.412, 82) :  20
(1166.515, 2) :  20
(1166.515, 10) :  20
(1166.515, 18) :  20
(1166.515, 29) :  20
(1166.515, 33) :  20
(1166.515, 52) :  20
(1166.515, 59) :  20
(1166.515, 62) :  20
(1166.515, 78) :  20
(1166.515, 79) :  20
(1166.515, 88) :  20
(1166.617, 19) :  20
(1166.617, 24) :  20
(1166.617, 31) :  20
(1166.617, 36) :  20
(1166.617, 38) :  20
(1166.617, 39) :  20
(1166.617, 40) :  20
(1166.617, 68) :  20
(1166.617, 69) :  20
(1166.72, 3) :  20
(1166.72, 4) :  20
(1166.72, 7) :  20
(1166.72, 8) :  20
(1166.72, 14) :  20
(1166.72, 15) :  20
(1166.72, 17) :  20
(1166.72, 23) :  20
(1166.72, 26) :  20
(1166.72, 27) :  20
(1166.72, 32) :  20
(1166.72, 41) :  20
(1166.72, 49) :  20
(1166.72, 50) :  20
(1166.72, 55) :  20
(1166.72, 60) :  20
(1166.72, 76) :  20
(1166.72, 77) :  20
(1166.72, 81) :  20
(1166.72, 83) :  20
(1166.777, 93) :  20
(1166.822, 6) :  20
(1166.822, 70) :  

(1317.288, 98) :  36
(1317.832, 65) :  36
(1317.832, 90) :  36
(1317.941, 42) :  36
(1318.159, 87) :  36
(1318.268, 91) :  36
(1318.548, 10) :  36
(1318.657, 6) :  36
(1318.657, 24) :  36
(1318.657, 52) :  36
(1318.766, 23) :  36
(1318.766, 32) :  36
(1318.766, 36) :  36
(1318.766, 68) :  36
(1318.766, 79) :  36
(1318.766, 100) :  36
(1318.875, 3) :  36
(1318.875, 8) :  36
(1318.875, 12) :  36
(1318.875, 31) :  36
(1318.875, 34) :  36
(1318.875, 47) :  36
(1318.875, 59) :  36
(1318.875, 61) :  36
(1318.875, 81) :  36
(1318.875, 88) :  36
(1318.965, 0) :  36
(1318.984, 2) :  36
(1318.984, 25) :  36
(1318.984, 35) :  36
(1318.984, 45) :  36
(1318.984, 48) :  36
(1318.984, 82) :  36
(1319.093, 27) :  36
(1319.093, 69) :  36
(1319.093, 73) :  36
(1319.093, 74) :  36
(1319.093, 80) :  36
(1319.093, 85) :  36
(1319.134, 95) :  36
(1319.202, 11) :  36
(1319.202, 14) :  36
(1319.202, 17) :  36
(1319.202, 19) :  36
(1319.202, 22) :  36
(1319.202, 26) :  36
(1319.202, 33) :  36
(1319.202, 44) : 

(1433.29, 15) :  45
(1433.29, 41) :  45
(1433.29, 49) :  45
(1433.29, 51) :  45
(1433.29, 55) :  45
(1433.404, 4) :  45
(1433.404, 7) :  45
(1433.404, 22) :  45
(1433.404, 29) :  45
(1433.404, 34) :  45
(1433.404, 35) :  45
(1433.404, 59) :  45
(1433.404, 63) :  45
(1433.404, 71) :  45
(1433.517, 10) :  45
(1433.517, 38) :  45
(1433.517, 40) :  45
(1433.517, 53) :  45
(1433.517, 61) :  45
(1433.517, 80) :  45
(1433.631, 25) :  45
(1433.631, 27) :  45
(1433.631, 33) :  45
(1433.631, 46) :  45
(1433.631, 72) :  45
(1433.631, 81) :  45
(1433.659, 93) :  45
(1433.744, 12) :  45
(1433.744, 19) :  45
(1433.744, 26) :  45
(1433.744, 60) :  45
(1433.744, 69) :  45
(1433.744, 77) :  45
(1433.744, 82) :  45
(1433.858, 8) :  45
(1433.858, 31) :  45
(1433.858, 36) :  45
(1433.858, 45) :  45
(1433.858, 52) :  45
(1433.858, 62) :  45
(1433.858, 75) :  45
(1433.858, 88) :  45
(1433.887, 92) :  45
(1433.887, 94) :  45
(1433.972, 23) :  45
(1433.972, 74) :  45
(1433.972, 78) :  45
(1433.972, 84) :  45


(1562.538, 39) :  54
(1562.538, 40) :  54
(1562.538, 41) :  54
(1562.538, 56) :  54
(1562.538, 57) :  54
(1562.538, 59) :  54
(1562.538, 66) :  54
(1562.538, 69) :  54
(1562.538, 76) :  54
(1562.538, 77) :  54
(1562.552, 92) :  54
(1562.657, 25) :  54
(1562.657, 28) :  54
(1562.657, 31) :  54
(1562.657, 33) :  54
(1562.657, 36) :  54
(1562.657, 53) :  54
(1562.657, 62) :  54
(1562.657, 63) :  54
(1562.657, 73) :  54
(1562.657, 75) :  54
(1562.657, 82) :  54
(1562.657, 84) :  54
(1562.657, 89) :  54
(1562.671, 94) :  54
(1562.671, 96) :  54
(1562.775, 9) :  54
(1562.775, 19) :  54
(1562.775, 46) :  54
(1562.775, 51) :  54
(1562.775, 54) :  54
(1562.775, 61) :  54
(1562.894, 10) :  54
(1562.894, 52) :  54
(1562.894, 60) :  54
(1562.894, 78) :  54
(1563.013, 26) :  54
(1563.013, 50) :  54
(1563.013, 55) :  54
(1563.131, 13) :  54
(1563.247, 101) :  54
(1563.497, 64) :  54
(1563.725, 6) :  54
(1563.725, 88) :  54
(1583.264, 21) :  55
(1583.384, 20) :  55
(1583.384, 43) :  55
(1583.384, 99)

...

(17529.229, 36) :  1265
(17529.229, 45) :  1265
(17529.627, 13) :  1265
(17529.627, 69) :  1265
(17530.425, 50) :  1265
(17532.02, 76) :  1265
(17532.02, 81) :  1265
(17532.419, 28) :  1266
(17532.419, 38) :  1266
(17534.014, 32) :  1266
(17534.396, 0) :  1266
(17534.413, 68) :  1266
(17535.21, 19) :  1266
(17535.609, 16) :  1266
(17536.806, 25) :  1266
(17537.603, 54) :  1267
(17537.603, 61) :  1267
(17537.878, 96) :  1267
(17538.401, 7) :  1267
(17540.794, 15) :  1267
(17541.167, 90) :  1267
(17541.193, 83) :  1267
(17542.266, 97) :  1267
(17543.56, 20) :  1268
(17544.783, 67) :  1268
(17545.955, 87) :  1268
(17546.379, 85) :  1268
(17547.152, 42) :  1268
(17547.152, 91) :  1268
(17547.576, 55) :  1268
(17549.546, 21) :  1269
(17549.969, 33) :  1269
(17551.541, 64) :  1269
(17552.363, 4) :  1269
(17553.138, 65) :  1269
(17553.936, 86) :  1269
(17554.757, 82) :  1270
(17555.156, 17) :  1270
(17555.555, 31) :  1270
(17555.555, 70) :  1270
(17555.954, 26) :  1270
(17555.954, 57) :  1270

(17934.683, 16) :  1333
(17935.157, 98) :  1333
(17935.49, 23) :  1333
(17935.49, 26) :  1333
(17935.49, 51) :  1333
(17939.524, 55) :  1334
(17940.331, 84) :  1334
(17941.137, 70) :  1334
(17942.681, 97) :  1334
(17943.558, 83) :  1334
(17944.295, 96) :  1334
(17944.768, 74) :  1335
(17945.979, 15) :  1335
(17946.313, 94) :  1335
(17946.455, 42) :  1335
(17946.455, 43) :  1335
(17947.189, 48) :  1335
(17947.189, 80) :  1335
(17948.4, 63) :  1335
(17948.803, 18) :  1335
(17948.803, 89) :  1335
(17949.61, 82) :  1335
(17950.014, 45) :  1336
(17950.014, 47) :  1336
(17951.224, 14) :  1336
(17951.299, 91) :  1336
(17952.106, 99) :  1336
(17953.646, 36) :  1336
(17955.192, 95) :  1337
(17956.471, 9) :  1337
(17958.489, 33) :  1337
(17958.489, 53) :  1337
(17960.103, 12) :  1337
(17960.103, 35) :  1337
(17960.507, 28) :  1338
(17961.718, 4) :  1338
(17962.121, 29) :  1338
(17962.525, 68) :  1338
(17962.929, 71) :  1338
(17965.351, 60) :  1338
(17965.754, 40) :  1339
(17966.637, 100) :  1339

(18372.539, 96) :  1407
(18372.957, 29) :  1407
(18372.957, 47) :  1407
(18373.365, 37) :  1407
(18373.365, 49) :  1407
(18374.59, 31) :  1407
(18375.815, 48) :  1407
(18377.448, 46) :  1408
(18379.49, 18) :  1408
(18379.49, 57) :  1408
(18380.306, 9) :  1408
(18384.39, 70) :  1409
(18386.024, 15) :  1409
(18387.431, 20) :  1409
(18388.066, 82) :  1409
(18388.474, 88) :  1409
(18390.101, 93) :  1410
(18390.108, 27) :  1410
(18390.517, 54) :  1410
(18391.108, 91) :  1410
(18391.326, 95) :  1410
(18392.967, 56) :  1410
(18393.376, 17) :  1410
(18393.784, 50) :  1410
(18394.193, 16) :  1410
(18394.193, 55) :  1410
(18394.602, 78) :  1410
(18396.236, 32) :  1411
(18396.42, 99) :  1411
(18398.278, 40) :  1411
(18398.278, 69) :  1411
(18398.687, 10) :  1411
(18400.73, 72) :  1411
(18400.73, 80) :  1411
(18401.955, 76) :  1412
(18402.773, 8) :  1412
(18402.773, 75) :  1412
(18403.59, 79) :  1412
(18405.224, 9) :  1412
(18406.042, 11) :  1412
(18406.042, 39) :  1412
(18406.042, 83) :  1412
(18

(18881.476, 30) :  1483
(18882.304, 52) :  1483
(18887.271, 15) :  1484
(18887.271, 28) :  1484
(18888.578, 93) :  1484
(18889.755, 33) :  1484
(18889.755, 78) :  1484
(18890.169, 18) :  1484
(18892.239, 74) :  1484
(18892.653, 82) :  1485
(18894.309, 19) :  1485
(18894.309, 58) :  1485
(18894.376, 97) :  1485
(18894.723, 72) :  1485
(18901.349, 79) :  1486
(18902.177, 70) :  1486
(18903.005, 5) :  1486
(18907.561, 52) :  1487
(18910.047, 31) :  1487
(18910.359, 64) :  1487
(18912.431, 98) :  1487
(18912.532, 16) :  1487
(18914.189, 49) :  1488
(18914.917, 91) :  1488
(18919.575, 25) :  1489
(18919.989, 68) :  1489
(18926.107, 21) :  1490
(18928.691, 19) :  1490
(18929.52, 51) :  1490
(18930.764, 26) :  1490
(18930.764, 58) :  1490
(18930.764, 75) :  1490
(18936.566, 10) :  1491
(18938.225, 45) :  1491
(18939.883, 30) :  1491
(18942.029, 97) :  1492
(18942.37, 31) :  1492
(18943.614, 29) :  1492
(18944.443, 16) :  1492
(18945.687, 8) :  1492
(18946.102, 28) :  1492
(18950.248, 80) :  1

(19500.321, 37) :  1565
(19500.321, 48) :  1565
(19500.321, 73) :  1565
(19500.741, 13) :  1565
(19500.741, 39) :  1565
(19500.782, 86) :  1565
(19500.782, 99) :  1565
(19500.901, 92) :  1565
(19500.901, 94) :  1565
(19501.162, 78) :  1565
(19501.162, 84) :  1565
(19501.322, 93) :  1565
(19501.583, 8) :  1565
(19501.583, 12) :  1565
(19501.583, 31) :  1565
(19501.583, 41) :  1565
(19501.583, 74) :  1565
(19501.624, 65) :  1565
(19502.045, 87) :  1565
(19502.045, 90) :  1565
(19502.045, 98) :  1565
(19502.065, 101) :  1565
(19502.465, 42) :  1565
(19503.005, 95) :  1565
(19503.266, 85) :  1565
(19503.728, 21) :  1566
(19504.718, 1) :  1566
(19504.99, 43) :  1566
(19504.99, 91) :  1566
(19505.832, 20) :  1566
(19508.506, 0) :  1566
(19523.044, 19) :  1567
(19529.943, 97) :  1567
(19532.805, 101) :  1567
(19533.989, 3) :  1567
(19534.41, 23) :  1567
(19534.831, 5) :  1567
(19534.831, 55) :  1567
(19534.831, 72) :  1567
(19535.302, 42) :  1568
(19535.673, 70) :  1568
(19537.358, 39) :  156